In [ ]:
import os
#import json
import glob
#import random
#import collections

import numpy as np
import pandas as pd
import pydicom
import cv2
#import matplotlib.pyplot as plt
#import scipy
from tqdm import tqdm

In [ ]:
train_df = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
train_df.head()

In [ ]:
train_df["MGMT_value"].value_counts()

In [ ]:
len(train_df["MGMT_value"])

In [ ]:
path = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/"

In [ ]:
def load_slices(patient, folder, p):
    path = p + patient + "/" + folder
    slices = [pydicom.dcmread(path + "/" + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    return slices

In [ ]:
def resize(scan, rm):
    scan[scan<-1000] = -1000
    scan[scan>400] = 400
    while rm > 0:
        dist = np.array([])
        for i in range(len(scan)-1):
            dist = np.append(dist,np.linalg.norm(scan[i]-scan[i+1]))
        idx = np.argmin(dist)
        if dist[idx] < 500:
            scan[idx] = (scan[idx] + scan[idx+1])/2
            scan = np.delete(scan,idx+1,axis=0)
            rm -= 1
        else:
            break
    while rm > 0:
        i = 0
        while i < len(scan)-1:
            if i%2 == 1:
                scan[i] = (scan[i] + scan[i+1])/2
                scan = np.delete(scan,i,axis=0)
                rm -= 1
            if rm == 0:
                break
            i += 1
    while rm < 0:
        i = 0
        while i < len(scan)-1:
            if i%2 == 1:
                img = (scan[i] + scan[i+1])/2
                scan = np.insert(scan,i+1,img,axis=0)
                rm += 1
            if rm == 0:
                break
            i += 1
    scan = scan - np.min(scan)
    if np.max(scan) != 0:
        scan = scan / np.max(scan)
    return scan

In [ ]:
'''total_data = {"FLAIR":[],"T1w":[],"T1wCE":[],"T2w":[]}
for _,patient in enumerate(tqdm(sorted(os.listdir(path)))):
    for key in total_data.keys():
        slices = load_slices(patient,key)
        image = np.stack([cv2.resize(s.pixel_array,(92,92)) for s in slices])
        #print("original:",image.shape)
        new_image = resize(image,len(slices)-40)
        #print("new:",new_image.shape)
        total_data[key].append(new_image)
#total_data["T2w"] = np.array(total_data["T2w"])'''

In [ ]:
'''from matplotlib import animation, rc
rc('animation', html='jshtml')


def create_animation(ims):
    fig,ax = plt.subplots(1,1,figsize=(6, 6))
    plt.axis('off')
    im = ax.imshow(ims[0], cmap="gray")
    
    def animate_func(i):
        ax.imshow(ims[i], cmap="gray")
        return [ims]
    return animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//24)
'''

In [ ]:
#create_animation(total_data["FLAIR"][-1])

In [ ]:
from tensorflow.keras.layers import Input, Conv3D, MaxPool3D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

#from keras.utils.np_utils import to_categorical

In [ ]:
keys = ["FLAIR","T1w","T1wCE","T2w"]

In [ ]:
models = {}

In [ ]:
for key in keys:
    input_layer = Input(shape = (40, 80, 80,1))
    x = Conv3D(filters = 64, kernel_size = 3, activation ='relu')(input_layer)
    x = Conv3D(filters = 64, kernel_size = 3, activation ='relu')(x)
    x = MaxPool3D(pool_size = 2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    x = Conv3D(filters = 128, kernel_size = 4,padding = 'Same', activation ='relu')(x)
    x = Conv3D(filters = 128, kernel_size = 4, activation ='relu')(x)
    x = MaxPool3D(pool_size = 2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Conv3D(filters = 256, kernel_size = 5,padding = 'Same', activation ='relu')(x)
    x = Conv3D(filters = 256, kernel_size = 5,padding = 'Same', activation ='relu')(x)
    x = MaxPool3D(pool_size = 2)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    
    x = Flatten()(x)
    x = Dense(units = 256, activation = 'relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(units = 1, activation = 'sigmoid')(x)

    model = Model(input_layer, output_layer)
    models[key] = model

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

early_stopping_cb = EarlyStopping(monitor="val_acc", patience=10)

In [ ]:
#train_pred = {}
#val_pred = {}
epochs = 30
for key in keys:
    X = []
    models[key].compile(
        loss="binary_crossentropy",
        optimizer= "adam",
        metrics=["acc"],
    )
    for _,patient in enumerate(tqdm(sorted(os.listdir(path)))):
        slices = load_slices(patient,key, path)
        image = np.stack([cv2.resize(s.pixel_array,(80,80)) for s in slices])
        #print("original:",image.shape)
        new_image = resize(image,len(slices)-40)
        del image
        #print("new:",new_image.shape)
        X.append(new_image)

    X_train, X_val, y_train, y_val = train_test_split(X, train_df["MGMT_value"], test_size = 0.15, random_state=42)
    X_train = np.array(X_train)
    X_val = np.array(X_val)
    y_train = np.array(y_train)
    y_val = np.array(y_val)
    print("model for: ",key)
    models[key].fit(X_train,y_train ,validation_data=(X_val,y_val), shuffle= True, epochs=epochs, batch_size=16,verbose=1, callbacks=[early_stopping_cb,learning_rate_reduction])
    #train_pred[key] = models[key].predict(X_train)
    #val_pred[key] = models[key].predict(X_val)
    del X
    del X_train
    del X_val
    del slices

In [ ]:
'''for key in keys:
    train_pred_c[key] = np.where(np.array(train_pred[key])>0.5,1,0)
    val_pred_c[key] = np.where(np.array(val_pred[key])>0.5,1,0)
train_pred_c = pd.DataFrame(train_pred_c)
val_pred_c = pd.DataFrame(val_pred_c)'''

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
'''for key in keys:
    print("key:",key," train:",accuracy_score(y_train,train_pred_c[key]), "val:",accuracy_score(y_val,val_pred_c[key]))
print("average: ","train:",accuracy_score(y_train, np.where(pd.DataFrame(train_pred_c).mean(axis=1).values>0.5,1,0)),
      "val:",accuracy_score(y_val,np.where(pd.DataFrame(val_pred_c).mean(axis=1).values>0.5,1,0)))'''

In [ ]:
test_path= "../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/"

In [ ]:
test = []
test_pred = {}
for key in keys:
    X = []
    for _,patient in enumerate(tqdm(sorted(os.listdir(test_path)))):
        slices = load_slices(patient,key,test_path)
        image = np.stack([cv2.resize(s.pixel_array,(80,80)) for s in slices])
        #print("original:",image.shape)
        new_image = resize(image,len(slices)-40)
        del image
        #print("new:",new_image.shape)
        X.append(new_image)
    test_pred[key] = models[key].predict(np.array(X))
    del X


In [ ]:
for key in keys:
    test_pred[key] = [x[0] for x in test_pred[key]]


In [ ]:
pred = pd.DataFrame(test_pred).mean(axis=1)
pred.head(),len(pred)

In [ ]:
test_pred = pd.DataFrame(test_pred)
submission = pd.DataFrame({"BraTS21ID":[int(id) for id in sorted(os.listdir(test_path))],
                           "MGMT_value":pred})

In [ ]:
submission

In [ ]:
submission.to_csv("submission.csv", index=False)